In [18]:
import json
#from openai import OpenAI
import requests
from transformers import pipeline
from pprint import pprint

In [14]:
def load_person_data():
    # Make API request
    person = []

    # Add first person data
    response = requests.get('https://random-person-generator.com/api')
    data = response.json()
    person.append({
        'name': data['identification']['full_name'],
        'dob': data['identification']['date_of_birth'],
        'email': data['contact_information']['email_address'],
        'service_num': data['financial_information']['cvv2'],
        'image': data['profile_photo']['256x256']
    })

    # Generate more data
    for i in range(10):
        response = requests.get('https://random-person-generator.com/api')
        data = response.json()

        person.append({
            'name': data['identification']['full_name'],
            'dob': data['identification']['date_of_birth'],
            'email': data['contact_information']['email_address'],
            'service_num': data['financial_information']['cvv2'],
            'image': data['profile_photo']['256x256']
        })

    return person  # Return the list directly

In [20]:
data = load_person_data()

In [24]:
data[1]

{'name': 'Afnan Miah',
 'dob': '01/23/1979',
 'email': 'a.miah@legacybeauty.net',
 'service_num': '094',
 'image': 'https://random-person-generator.com/storage/images/profile_photos/v1/256x256/0e5fa792-8485-4ab9-86d6-172bcd5e8537.jpg'}

In [27]:
# Load the text generation model
model = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

def find_matching_person(name=None, image_url=None):
    person_data = data

    if name or image_url:
        if name:
            # Prepare a query for the model (optional, depending on your needs)
            query = f"Find a person with the name '{name}'."
            response = model(query, max_length=500, truncation=True)
            result = response[0]['generated_text']

        # Check for matches in the person data
        matches = []
        for person in person_data:
            if (name and name.lower() in person['name'].lower()) or (image_url and person['image'] == image_url):
                matches.append(person)

        return matches  # Return all matching profiles

    return None

In [28]:
# Main execution
if __name__ == "__main__":
    # Load the text generation model
    model = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

    # Input name or image URL from the user
    user_input = input("Enter the person's name or image URL: ")

    # Check if the input is a URL
    if user_input.startswith("http://") or user_input.startswith("https://"):
        profile = find_matching_person(image_url=user_input)
    else:
        profile = find_matching_person(name=user_input)

    if profile:
        print("Matching Profile Found:")
        pprint(json.dumps(profile, indent=2))
    else:
        print("No matching profile found.")

Enter the person's name or image URL: Af


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Matching Profile Found:
('[\n'
 '  {\n'
 '    "name": "Afnan Miah",\n'
 '    "dob": "01/23/1979",\n'
 '    "email": "a.miah@legacybeauty.net",\n'
 '    "service_num": "094",\n'
 '    "image": '
 '"https://random-person-generator.com/storage/images/profile_photos/v1/256x256/0e5fa792-8485-4ab9-86d6-172bcd5e8537.jpg"\n'
 '  }\n'
 ']')
